Analysis of Estimation Error
=======================

**Requirements**:
- The [evo](https://github.com/MichaelGrupp/evo) Python package
    - Can be installed with `pip2 install evo -U --user`
- Python 2 kernel (for `import rosbag`)
- make sure you enabled interactive widgets via:
    ```
    jupyter nbextension enable --py widgetsnbextension
    ```
- rosbags with `geometry_msgs/PoseStamped`, `geometry_msgs/TransformStamped`, `geometry_msgs/PoseWithCovarianceStamped` or `nav_msgs/Odometry` topics.

*Based on evo's [`metrics_interactive`](https://github.com/MichaelGrupp/evo/tree/master/notebooks) notebook for interactive widgets.*

In [1]:
from __future__ import print_function

from evo.tools import log
log.configure_logging()

Initialized new /home/plusk01/.evo/settings.json


In [2]:
from evo.tools import plot
from evo.tools.plot import PlotMode
from evo.core.metrics import PoseRelation, Unit
from evo.tools.settings import SETTINGS

# temporarily override some package settings
SETTINGS.plot_figsize = [6, 6]
SETTINGS.plot_split = True
SETTINGS.plot_usetex = False

# magic plot configuration
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib notebook

In [3]:
# interactive widgets configuration
import ipywidgets

check_opts_ape = {"align": True, "correct_scale": False, "show_plot": True}
check_boxes_ape=[ipywidgets.Checkbox(description=desc, value=val) for desc, val in check_opts_ape.items()]
check_opts_rpe = {"align": True, "correct_scale": False, "all_pairs": False, "show_plot": True}
check_boxes_rpe=[ipywidgets.Checkbox(description=desc, value=val) for desc, val in check_opts_rpe.items()]
delta_input = ipywidgets.FloatText(value=1.0, description='delta', disabled=False, color='black')
du_selector=ipywidgets.Dropdown(
    options={u.value: u for u in Unit},
    value=Unit.frames, description='delta_unit'
)
pm_selector=ipywidgets.Dropdown(
    options={p.value: p for p in PlotMode},
    value=PlotMode.xy, description='plot_mode'
)
pr_selector=ipywidgets.Dropdown(
    options={p.value: p for p in PoseRelation},
    value=PoseRelation.translation_part, description='pose_relation'
)

## Load Trajectories

In [4]:
from evo.core.trajectory import PoseTrajectory3D

def trimTrajectory(traj, m, n):
    ps = traj.positions_xyz[m:-n]
    qs = traj.orientations_quat_wxyz[m:-n]
    ts = traj.timestamps[m:-n]

    return PoseTrajectory3D(ps, qs, ts, meta=traj.meta)

In [58]:
import numpy as np
from evo.core import lie_algebra as lie
from evo.core import transformations as tr

def poseCMA(m, T, Tmeas):
    p, q = T
    pmeas, qmeas = Tmeas
    
    p = 1/(m+1) * ( m*p + pmeas )
    q = tr.quaternion_slerp(q, qmeas, 1/(m+1))
    
    

def registerCoordinateFrames(trajW, trajV, n=200):
    '''
    VICON and VISLAM (should) describe the same dynamical motion,
    albeit in different frames. The
    '''
    
    # T_WV is the pose of the VISLAM coordinate system w.r.t.
    # the world (VICON), which we initialize as identity.
    T_WV = (np.array([1., 0., 0., 0.]), np.array([0., 0., 0.]))
    
    # calculate a cumulative moving average filter on T_WV
    # x_n+1 = 1/(n+1) [ n*x_n + newMeas ]
    
    for i in range(n):
        # body w.r.t world (VICON)
        T_WB = (trajW.orientations_quat_wxyz[i], trajW.positions_xyz[i])
        
        # body w.r.t VISLAM
        T_VB = (trajV.orientations_quat_wxyz[i], trajV.positions_xyz[i])
        
        # calculate relative transformation of VISLAM w.r.t world (VICON)
        q_WV = np.array([1., 0., 0., 0.])
        p_WV = np.array([0., 0., 0.])
        T_WV_measurement = (q_WV, p_WV)
        
        # update the cumulative moving average of the pose
        T_WV = poseCMA(i, T_WV, T_WV_measurement)
    
    return T_WV

In [23]:
# import rosbag
from evo.tools import file_interface
from evo.core import sync

# # Load bag with all topics
# bag = '/home/plusk01/Documents/bags/hx01/vio_test_2.bag'
# bag_handle = rosbag.Bag(bag)

# load ORB_SLAM3 trajectory
traj_orbslam = file_interface.read_tum_trajectory_file("/home/plusk01/.ros/orbslam3_traj_tum.txt")

# load ground truth trajectory
traj_truth = file_interface.read_tum_trajectory_file("/home/plusk01/.ros/truth_traj_tum.txt")

# # Trim trajectories
# n = 200
# # traj_vicon = trimTrajectory(traj_vicon, n)
# traj_vislam = trimTrajectory(traj_vislam, 300, 300)

# time sync the trajectories
traj_ref, traj_est = sync.associate_trajectories(traj_truth, traj_orbslam)

In [24]:
print(traj_ref)
print(traj_est)

7281 poses, 247.719m path length, 279.860s duration
7281 poses, 487.048m path length, 279.861s duration


## Absolute Pose Error (APE)

Also known as *Absolute Trajectory Error* (ATE).

In [25]:
import evo.main_ape as main_ape
import evo.common_ape_rpe as common
import copy

count = 0
results = []

def callback_ape(pose_relation, align, correct_scale, plot_mode, show_plot):
    global results, count
    est_name="APE Test #{}".format(count)
    
    traj_ref1 = copy.deepcopy(traj_ref)
    traj_est1 = copy.deepcopy(traj_est)
    
    result = main_ape.ape(traj_ref1, traj_est1, est_name=est_name,
                          pose_relation=pose_relation, align=align, correct_scale=correct_scale)
    count += 1
    results.append(result)
    
    if show_plot:
        fig = plt.figure()
        ax = plot.prepare_axis(fig, plot_mode)
        plot.traj(ax, plot_mode, traj_ref, style="--", alpha=0.5)
        plot.traj_colormap(
            ax, result.trajectories[est_name], result.np_arrays["error_array"], plot_mode,
            min_map=result.stats["min"], max_map=result.stats["max"])

_ = ipywidgets.interact_manual(callback_ape, pose_relation=pr_selector, plot_mode=pm_selector,
                               **{c.description: c.value for c in check_boxes_ape})

interactive(children=(Dropdown(description='pose_relation', index=1, options={'full transformation': <PoseRela…

## Relative Pose Error (RPE)

Also known as *Relative Trajectory Error* (RTE).

In [12]:
import evo.main_rpe as main_rpe

count = 0
results = []

def callback_rpe(pose_relation, delta, delta_unit, all_pairs, align, correct_scale, plot_mode, show_plot):
    global results, count
    est_name="RPE Test #{}".format(count)
    result = main_rpe.rpe(traj_ref, traj_est, est_name=est_name,
                          pose_relation=pose_relation, delta=delta, delta_unit=delta_unit, 
                          all_pairs=all_pairs, align=align, correct_scale=correct_scale, 
                          support_loop=True)
    count += 1
    results.append(result)
    
    if show_plot:
        fig = plt.figure()
        ax = plot.prepare_axis(fig, plot_mode)
        plot.traj(ax, plot_mode, traj_ref, style="--", alpha=0.5)
        plot.traj_colormap(
            ax, result.trajectories[est_name], result.np_arrays["error_array"], plot_mode,
            min_map=result.stats["min"], max_map=result.stats["max"])

_ = ipywidgets.interact_manual(callback_rpe, pose_relation=pr_selector, plot_mode=pm_selector, 
                               delta=delta_input, delta_unit=du_selector, 
                               **{c.description: c.value for c in check_boxes_rpe})

interactive(children=(Dropdown(description='pose_relation', index=1, options={'full transformation': <PoseRela…